In [1]:
from __future__ import unicode_literals
from xml.etree import ElementTree as Xet
import csv
import pandas as pd
import urllib3
import requests
#pip install requests beautifulsoup4 
import bs4
from bs4 import BeautifulSoup as bs

#https://github.com/alexmercerind/youtube-search-python
#pip install youtube-search-python
import youtubesearchpython as yts

In [ ]:
#Module 1: Playing with html pages
#converts the xml page into a dataframe, filters based on watching/completed and also adds mal link in last column
def MakeCSV(exportedxml):
    xml = Xet.parse(exportedxml)
    csvfile = open('mal_to_csv.csv','w',encoding='utf-8')
    csvfile_writer = csv.writer(csvfile)
    csvfile_writer.writerow(['AnimeTitle','ID','Status'])

    for anime in xml.findall('anime'):
        if(anime):
            animename = anime.find('series_title')
            animeid = anime.find('series_animedb_id')
            animestatus = anime.find('my_status')
            csv_line = [animename.text, animeid.text, animestatus.text]
            csvfile_writer.writerow(csv_line)
    csvfile.close()  

    df = pd.read_csv('mal_to_csv.csv')
    df2= df.dropna()
    filters = ['Completed', 'Watching']
    df3 = df2[df2['Status'].isin(['Completed', 'Watching'])].reset_index()
    del df3['index']
    def make_url(row):
        return 'https://myanimelist.net/anime/' + str(row[1])
    df3['URL'] = df3.apply(make_url, axis = 1)
    return df3

#this function gives the title, season and year of the anime as a list
def AnimeAndSeason(entry_page):
    title  = entry_page.title.text.replace('\n', ' ').strip(' MyAnimeList.net')
    title  = title.strip(' -')
    links  = entry_page.select('span > a')
    years  = range(1900,2050)
    season = ''
    year   = 1900
    for row in links:
        test = row.contents[0].split()
        if len(test) == 2 and test[1].isdigit() and int(test[1]) in years:
            season_info = test
            season = season_info[0] + ' ' + season_info[1]
            year = int(season_info[1])
    return(title, season, year)

#function below gives a lil bit fucked list of the category/song/artist for individual mal page
def SongAndArtist(entry_page):
    testing = entry_page.find_all('div', attrs={'class':'di-tc va-t'})
    testing2 = entry_page.find_all('div', attrs={'class':'di-tc va-t borderDark pb4'})
    testing3 = testing + testing2
    SAAlist = []
    for entry in testing3:
        category = entry.h2
        if type(category) == bs4.element.Tag: 
            category_name = category.contents
            SAAlist.append(category_name)

        song_td_set = entry.find_all('td', {'width':'84%'})
        for song_td in song_td_set:
            song_data = song_td.text.replace('\n', ' ').strip()
            row = song_data.split('\xa0') #split song# and song-artist
            for datas in row:
                if ' by ' in datas:
                    row2 = datas.split(' by ') #split song and artist
                    data_entry = [row2[0].strip('\"'), row2[1]]  #join song, artist
                    song_links = song_td.find_all('input')
                    for link in song_links:
                        somelink = link['value']
                        if 'spotify' in somelink:
                            data_entry.append(somelink)
                    SAAlist.append(data_entry)
    return SAAlist

#this combines the previous 2 functions and gives a nice dataframe of each row having all the identifying factors of a song
def CreateSongDF(entry_page):
    aas = AnimeAndSeason(entry_page)
    saa = SongAndArtist(entry_page)
    if aas[0] == 'Gyo (GYO: Tokyo Fish Attack!)': saa[2][0] = 'End Theme'
        
    for line in saa:
        if 'Opening Theme' in line: op_index = saa.index(line)
        if 'Ending Theme' in line: ed_index = saa.index(line)
            
    OP = []
    category = saa[op_index][0]
    for op in range(op_index+1,ed_index):
        temp = saa[op]
        temp.insert(0,category)
        OP.append(temp)

    ED = []
    category = saa[ed_index][0]
    for ed in range(ed_index+1,len(saa)):
        temp = saa[ed]
        temp.insert(0,category)
        ED.append(temp)
    
    def Clean_Blanks(List, Length):
        for item in List:
            if len(item) != Length:
                item.extend(' ')
       
    Clean_Blanks(ED, 4)
    Clean_Blanks(OP, 4)
    song_columns = ['category', 'songname', 'artist', 'link']
    opdb = pd.DataFrame(OP, columns = song_columns)
    eddb = pd.DataFrame(ED, columns = song_columns)
    SongDB = pd.concat([opdb,eddb], axis = 0, ignore_index = True)
    SongDB['extra'] = ''
    return SongDB

#Module 2: Playing with internet for spotify and youtube links
#this creates a new dataframe (withc columns as song metadata and a 'links' column) from the csv containing mal urls
def GetSpotify(malcsv):
    song_table = pd.read_csv(malcsv)
    song_table = song_table.reset_index()
    del song_table['Unnamed: 0']
    del song_table['index']
    link_table = pd.DataFrame(columns = ['anime', 'season', 'year','category', 'songname', 'artist', 'link', 'extra'])
    for i in song_table.index:
        anime_link = song_table.loc[i,'URL']
        html       = requests.get(anime_link).text #type: ignore       
        mal_page   = bs(html, 'lxml')  
        aas        = AnimeAndSeason(mal_page)   

        song_subtable    = CreateSongDF(mal_page)
        anime_subtable   = pd.DataFrame(columns=['anime', 'season', 'year'])
        print('Fetching spotify links for', aas[0])
        
        for i in range(0, len(song_subtable)):
            anime_subtable.loc[i,'anime']  = aas[0]
            anime_subtable.loc[i,'season'] = aas[1]
            anime_subtable.loc[i,'year']   = aas[2]
        anime_subtable = pd.concat([anime_subtable, song_subtable], axis = 1)
        link_table     = pd.concat([link_table, anime_subtable], axis = 0)
    link_table = link_table.reset_index()
    del link_table['index']
    return link_table

#update the spotify sheet with youtube links as well to get a final sheet with all links
def GetYoutube(spotcsv):
    links.to_csv(spotcsv)
    del link_table['Unnamed: 0']
    f=1
    for j in link_table.index:
        if 'spotify' in str(link_table.loc[j,'link']): continue
        if 'youtube' in str(link_table.loc[j,'link']): continue 

        print(j,'Fetching youtube links for', link_table.loc[j,'anime'],link_table.loc[j,'songname'])
        search = link_table.loc[j,'songname'] + ' ' + link_table.loc[j,'artist']
        
        if link_table.loc[j,'extra'] == 'fuckup': search = link_table.loc[j,'anime'] + ' ' + link_table.loc[j,'songname']
        if link_table.loc[j,'extra'] == 'fuckup2': search = link_table.loc[j,'songname']

        first_result = yts.VideosSearch(search, limit = 1).result()
        if first_result['result'] == []: 
            if link_table.loc[j,'extra'] == 'fuckup': link_table.loc[j,'extra'] = 'fuckup2'
            link_table.loc[j,'extra'] = 'fuckup'
            print('youtube search fuckup #' + str(f))
            f+=1
            continue
            link_table.loc[j,'link'] =  first_result['result'][0]['link']
            link_table.loc[j,'extra'] =  first_result['result'][0]['title']
    link_table['status'] = ''
    return link_table

#function to make all entries in the databases valid filenames
def ValidFile(txt):
    filename = ''
    if '...' in txt:
        txt = txt.strip('...')
    for i in txt:
        #if i == '＋': #for 22/7 ova
            #filename = filename + ' plus '
        if i not in '\/*?<>|"':
            filename = filename + i
        elif i in ':':
            filename = filename + '-'
        else:
            filename = filename + '_'
    return filename
    

In [217]:
#new = pd.read_csv('song_link_sheet_new.csv')
old = pd.read_csv('song_link_sheet.csv')
del old['Unnamed: 0']
new = old.head(20)
old = old.head(20)

#new = pd.read_csv('song_link_sheet_new.csv')
old = pd.read_csv('song_link_sheet.csv')
del old['Unnamed: 0']
new = old.head(20)
old = old.head(20)

new.loc[-1] = old.loc[3]
new.loc[-2] = old.loc[16]
for i in range(26):
    new.index = new.index+1
    new = new.sort_index()
new.loc[24,'artist'] = 'oongy noongy'
new.loc[24,'link'] = 'https://open.spotify.com/track/1ZJreL7IUwnJrBigeIBvD0'
new.loc[24,'season'] = 'Fall 2023'
new.loc[24,'year'] = 2023
new.loc[24,'anime'] = "your mom is my gf"

new.loc[25,'artist'] = 'dj astarfigullah'
new.loc[25,'link'] = 'https://www.youtube.com/watch?v=fDSm1t4kgzk'
new.loc[25,'season'] = 'Spring 2024'
new.loc[25,'year'] = 2024
new.loc[25,'anime'] = "your mom is my gf season 2"

new = new.reset_index()
del new['index']
for i in new.index:
    new.loc[i, 'status'] = ''
for i in old.index:
    old.loc[i,'status'] = 'downloaded'

new.loc[11, 'link'] = 'https://open.spotify.com/track/1sfdgssrsdvvD0'
new.loc[11, 'extra'] = ''

#added (one yt and 1 spot)at 0 and 1, updated yt to spot at 11 
temp = pd.DataFrame(columns = old.columns.tolist())


found a new link: https://open.spotify.com/track/1ZJreL7IUwnJrBigeIBvD0
found a new link: https://www.youtube.com/watch?v=fDSm1t4kgzk
found a new link: https://open.spotify.com/track/1sfdgssrsdvvD0


,anime,season,year,category,songname,artist,link,extra,status
0,your mom is my gf,Fall 2023,2023,Opening Theme,Muzui (ムズイ),oongy noongy,https://open.spotify.com/track/1ZJreL7IUwnJrBi...,NaN,
1,your mom is my gf season 2,Spring 2024,2024,Ending Theme,Subversive,dj astarfigullah,https://www.youtube.com/watch?v=fDSm1t4kgzk,NaN,
2,22/7,Winter 2020,2020,Ending Theme,Ikiru koto ni Raku ni Naritai (生きることに楽になりたい),Sakura Fujima (Sally Amaki),https://open.spotify.com/track/1sfdgssrsdvvD0,yt upgrade to spot,


In [232]:
new

,anime,season,year,category,songname,artist,link,extra,status
0,your mom is my gf,Fall 2023,2023,Opening Theme,Muzui (ムズイ),oongy noongy,https://open.spotify.com/track/1ZJreL7IUwnJrBi...,NaN,
1,your mom is my gf season 2,Spring 2024,2024,Ending Theme,Subversive,dj astarfigullah,https://www.youtube.com/watch?v=fDSm1t4kgzk,NaN,
2,100-man no Inochi no Ue ni Ore wa Tatteiru,Fall 2020,2020,Opening Theme,Anti world,Kanako Takatsuki,https://open.spotify.com/track/47mtXCSExRrKA9c...,NaN,
3,100-man no Inochi no Ue ni Ore wa Tatteiru,Fall 2020,2020,Ending Theme,Carpe・Diem (カルペ・ディエム),Liyuu,https://open.spotify.com/track/2MtalMSV45ThxAF...,NaN,
4,100-man no Inochi no Ue ni Ore wa Tatteiru 2nd...,Summer 2021,2021,Opening Theme,Baddest,Kaede Higuchi (樋口楓),https://open.spotify.com/track/0XMsMnaW0zceeFu...,NaN,
5,100-man no Inochi no Ue ni Ore wa Tatteiru 2nd...,Summer 2021,2021,Ending Theme,Subversive,Kanako Takatsuki (高槻かなこ),https://open.spotify.com/track/1zJreL7IUwnJRBi...,NaN,
6,2.43: Seiin Koukou Danshi Volley-bu,Winter 2021,2021,Opening Theme,Mahi (麻痺),yama,https://open.spotify.com/track/1LwSnnsoKcAUv9T...,NaN,
7,2.43: Seiin Koukou Danshi Volley-bu,Winter 2021,2021,Ending Theme,Undulation,Soushi Sakiyama (崎山蒼志),https://open.spotify.com/track/3WcLxnWX9JXQjT5...,NaN,
8,22/7,Winter 2020,2020,Opening Theme,Muzui (ムズイ),22_7,https://open.spotify.com/track/6WgPTB2vJe3BuRI...,NaN,
9,22/7,Winter 2020,2020,Ending Theme,Sora no Emerald (空のエメラルド),22_7,https://open.spotify.com/track/1wPwvkMMClSdTm3...,NaN,


In [ ]:
'''
#######################
tables = testing2[0].find_all('td', {'width':'84%'})
tablerows = tablerows.find_all('span ', attrs = {'class':'theme-song-index'})

headings = []
for td in tablerows[0].find_all('td'):
    #remove any newlines and extra spaces from left and right
    headings.append(td.b.text.replace('\n', ' ').strip())
#print(headings)

a=tables[1].next.next.next
a
#######################
Songsettitle = testing2[2].find_all('span',{'class':'theme-song-title'})
outputSongsetartist = testing2[2].find_all('span',{'class':'theme-song-artist'})

#https://stackoverflow.com/questions/49515530/how-to-convert-bs4-element-tag-to-pandas
sst = pd.Series([i.text.strip('\"') for i in Songsettitle])
ssa = pd.Series([i.text for i in Songsetartist])

#https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
Songset = pd.concat([sst,ssa], axis=1)
Songset
#######################
text = []
for x in tables[1]:
    if isinstance(x, bs4.element.NavigableString):
        text.append(x.string.strip())
print(' '.join(text))
text
text[0].strip('\"')
#######################
soup = bs(
    <html>
        <h2> Heading 2 <h2> Heading 1 </h2></h2>
        <h2> Heading 1 </h2>
    </html>, 'lxml')
# Get the whole h2 tag
tag = soup.h2
# Get the string inside the tag and convert 
# it into string
text = []
for x in tag:
    if isinstance(x, bs4.element.NavigableString):
        text.append(x.strip())
print(' '.join(text))
text
#######################
y = entry_page.find_all('div', {'class':'spaceit_pad'})
for spaceit in y:
    darktext = spaceit.find_all('span', {'class':'dark_text'})
    for entry in darktext:
        if entry.contents == 'Premiered:':
        print(entry)
#######################
for i in saa:
    if 'https' in i:
        print(1)
    else:
        print(temp)
        temp = []
    temp.extend(i)
#######################    
animelink = 'https://myanimelist.net/anime/37206'
html = requests.get(animelink).text
entry_page = bs(html, 'lxml')

for i in filtered_mal.index:
    if filtered_mal['AnimeTitle'][i] == 'Gyo':
        print(i)
    
filtered_mal.loc[269]

for i in range(1206,1241):
    #with_spotify['anime'][i] = 'M' + with_spotify['anime'][i]
    #with_spotify['anime'][i] = with_spotify['anime'][i].lstrip('A')
    print(with_spotify['anime'][i])
#######################
i = 0        
year   = songset2.loc[i,'year']
season = songset2.loc[i,'season']
anime  = songset2.loc[i,'anime']
oped   = songset2.loc[i,'category']
title  = songset2.loc[i,'songname']
artist = songset2.loc[i,'artist']
path   = str(year) + '/' + season + '/' + anime + '/' + oped + '/' + artist + '/'+ title

first_result = yts.VideosSearch(title + ' ' + artist, limit = 2)
yt_link      = first_result.result()['result'][0]['link']

yt = YouTube(yt_link)
audio = yt.streams.get_audio_only()
print(yt.title, yt.thumbnail_url)

try: 
    audio.download(output_path = path) 
except: 
    print('Download Error) 
    
file_name = audio.default_filename
#source = path + '/' + file_name
#if ' ' in file_name:
#    os.rename(source, source.replace(' ', '_'))
#file_name = source.replace(' ','_')

file_without_ext = os.path.splitext(file_name)[0]
command = f'ffmpeg -i {file_name} {file_without_ext}.mp3'
os.system(command)
#######################
songset2 = with_spotify
pd.set_option('display.max_columns', 7)
pd.reset_option('max_columns')
songset2
#######################
x = pd.DataFrame({'hello': [1,2,3],
                'bye':[4,5,6]})
print(x)
for j  in range(x.ndim):
    for i in range(len(x)):
        x.iloc[i,j] = "BOOM"

print(x)
#######################
#initializing 2 dataframes, 1 with spotify links and 1 without
sheet = pd.read_csv('nice_looking_song_sheet.csv')
del sheet['Unnamed: 0']
sws = sheet
for i in sheet.index:
    sws.loc[i,'anime']    = ValidFile(sws.loc[i,'anime'])
    sws.loc[i,'songname'] = ValidFile(sws.loc[i,'songname'])
    sws.loc[i,'artist']   = ValidFile(sws.loc[i,'artist'])
no_spotify   = pd.DataFrame(columns = ['anime', 'season', 'year', 'category', 'songname', 'artist'])
with_spotify = pd.DataFrame(columns = ['anime', 'season', 'year', 'category', 'songname', 'artist', 'spotify'])

for i in sheet.index:
    if sheet['spotify'][i] == ' ': no_spotify.loc[i] = sheet.loc[i]
    else: with_spotify.loc[i] = sheet.loc[i]

with_spotify = with_spotify.reset_index()
del with_spotify['index']
with_spotify['status'] = ''
no_spotify = no_spotify.reset_index()
del no_spotify['index']
no_spotify['link'] = None
#with_spotify.to_csv('with_spotify.csv')
#no_spotify.to_csv('no_spotify.csv')
#######################
    if link_table.loc[j,'songname'] == 'We☆Pace!!':
        link_table.loc[j,'link'] =  'https://www.youtube.com/watch?v=qdfBjYmG9a0'
        link_table.loc[j,'extra'] =  'manually entered cuz fuck me'
        continue
#######################
new = pd.read_csv('song_link_sheet.csv')
old = pd.read_csv('with_spotify.csv')
del new['Unnamed: 0']
del old['Unnamed: 0']

for n in new.index:
    if n > 500: break
    for o in old.index:
        if o > 400: break
        spot = old.loc[o, 'spotify']
        status = old.loc[o,'status']
        if new.loc[n, 'link'] == spot:
            new.loc[n, 'status'] = status

'''